In [21]:
from transformers import pipeline
from collections import defaultdict

# Replace this with your own checkpoint
model_checkpoint = "../models/epoch_30"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)

In [22]:
def preprocess_description(description):
    single_description = description.strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('A' <= letter and letter <= 'Z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(
                single_description[last_special + 1:idx + 1].strip())
    return " ".join(new_description)

In [30]:
import json

# Replace this description
description = '''
【Easy to Install】: Christmas lights hanging hooks 20-Pack are quick and easy to install, You can hit the wood directly with a hammer, or you can screw in the wood with a wrench. To install a hook on the wall, you need to make a hole in the wall, put a plastic tube, and then put the hook into the plastic tube. 1/8in drill bit is recommended.
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.6:
                high_score_ans[group].add(res['word'].lower().strip())
                
new_high_score_ans = defaultdict(list)
for key_dict in high_score_ans.keys():
    new_high_score_ans[key_dict] = list(high_score_ans[key_dict])

print("-"*100)            
print(json.dumps(new_high_score_ans, sort_keys=True, indent=4))                

【 Easy to Install 】 : Christmas lights hanging hooks 20 - Pack are quick and easy to install , You can hit the wood directly with a hammer , or you can screw in the wood with a wrench . To install a hook on the wall , you need to make a hole in the wall , put a plastic tube , and then put the hook into the plastic tube . 1 / 8in drill bit is recommended . 
----------------------------------------------------------------------------------------------------
{
    "DIMENSION": [
        "1 / 8in"
    ],
    "MAT": [
        "wood",
        "plastic"
    ],
    "PROPERTY": [
        "20 - pack",
        "drill bit"
    ]
}
